# A regression model for California Housing Prices Dataset

To replicate with success the experiments conducted in the paper we tried, as mush as possible, to replicate also the model used by authors:
"We used a 8:2 train-test split and trained a regression model with 3 hidden layers with 168K parameters,
using Adam optimizer minimizing MSE for 200 epochs"

1) Import of necessary libraries and California Housing dataset fetching function:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tqdm import tqdm
from sklearn.datasets import fetch_california_housing
from torchmetrics import ExplainedVariance

2) Setup to enable saving checkpoints in a dedicated folder, to reuse them in the implementation of TracInCP and in other eventualities:

In [3]:
def savecheckpoint(checkpoint, filename="chpcheckpoint.pth.tar", dir_name="batch"):
  print("=> saving checkpoint")
  ph = F"./{dir_name}/{filename}"
  torch.save(checkpoint, ph)

3: Definition of the model: to replicate the results of the paper, a 3 hidden layer neural network has been defined with 164K parameters.

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(8, 400)
        self.fc2 = nn.Linear(400, 400)
        self.fc3 = nn.Linear(400, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the network
model = Net()

# Count the total number of parameters in the network
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Parameters:", total_params)

Total Parameters: 164401


5) Dataset is fetched and separated into features X and labels y:

In [4]:
data = fetch_california_housing()
X = data.data.astype(np.float32)
y = data.target

5) Then it is further processed: first the features are standardized with a standard scaler then they are divided into Train and Test sets. The trade off between train and test is set to 8:2 as in the paper.

In [5]:
# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

5) The splitted sets are processed to convert them into tensors and batched. The model is loaded into CUDA if it is available, else it will run on cpu.

In [6]:
# Convert numpy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float)
X_test = torch.tensor(X_test, dtype=torch.float)
Y_train = torch.tensor(Y_train, dtype=torch.float).view(-1, 1)
Y_test = torch.tensor(Y_test, dtype=torch.float).view(-1, 1)

datasets = torch.utils.data.TensorDataset(X_train, Y_train)
datasets_test = torch.utils.data.TensorDataset(X_test, Y_test)

batch_size = 8

train_iter = torch.utils.data.DataLoader(datasets, batch_size=batch_size, shuffle=True)
test_iter = torch.utils.data.DataLoader(datasets_test, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

# Create an instance of the model
model = Net()
model.to(device)

cuda


Net(
  (fc1): Linear(in_features=8, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=1, bias=True)
  (relu): ReLU()
)

6) Training: in order to replicate also here the process of the paper, an Adam optimizer is exploited and the network is trained for 200 epochs.

In [7]:
import os
if not os.path.exists("CHP_checkpoints/batch_"+str(batch_size)):
    os.makedirs("CHP_checkpoints/batch_"+str(batch_size))

# Define the loss function and optimizer

loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_list = []

num_epochs = 200

for epoch in tqdm(range(num_epochs)):
    if epoch % 10 == 0:
        savecheckpoint(model.state_dict(), str(epoch) + "chpcheckpoint.pth.tar", "CHP_checkpoints/batch_"+str(batch_size))
    for x, y in train_iter:
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        l = loss(output, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    loss_list.append(l.data)
    print("epoch {} loss: {:.4f}".format(epoch + 1, l.item()))

  0%|                                                   | 0/200 [00:00<?, ?it/s]

=> saving checkpoint


  0%|▏                                          | 1/200 [00:12<40:55, 12.34s/it]

epoch 1 loss: 0.1048


  1%|▍                                          | 2/200 [00:15<22:36,  6.85s/it]

...

epoch 198 loss: 0.0322


100%|████████████████████████████████████████▊| 199/200 [09:42<00:02,  2.83s/it]

epoch 199 loss: 0.6919


100%|█████████████████████████████████████████| 200/200 [09:45<00:00,  2.93s/it]

epoch 200 loss: 0.1518


7) Results: the model built achieved an explained variance of 0.72 on test set and 0.93 on train set, which is very similar to the variances obtained in the paper (respectively 0.72 and 0.70).
<p align = "center">
$$Explained Variance = 1 - \frac{Var(y - \hat{y})}{Var(y)}$$
</p>


In [27]:
# Evaluate the model
model.eval()
with torch.no_grad():
    X_test = X_test.to(device)
    Y_test = Y_test.to(device)
    predicted = model(X_test)
    mse = loss(predicted, Y_test)
    mae = torch.mean(torch.abs(predicted - Y_test))

    explained_variance = ExplainedVariance()
    exp_var_test = explained_variance(predicted, Y_test)

    X_train = X_train.to(device)
    Y_train = Y_train.to(device)
    predicted_train = model(X_train)

    exp_var_train = explained_variance(predicted_train, Y_train)

    print("Mean Squared Error: {:.4f}".format(mse))
    print("Mean Absolute Error: {:.4f}".format(mae))
    print("Explained Variance on test set: {:.4f}".format(exp_var_test))
    print("Explained Variance on train set: {:.4f}".format(exp_var_train))

Mean Squared Error: 0.3043
Mean Absolute Error: 0.3598
Explained Variance on test set: 0.7678
Explained Variance on train set: 0.9305
